In [1]:
from whoosh.index import create_in, open_dir
from whoosh.fields import *
schema = Schema(
    title=TEXT(stored=True),
    Id=ID(stored=True, unique=True), 
    content=TEXT(stored=True), 
    author=TEXT(stored=True), 
    date=DATETIME(stored=True), 
    has_image=BOOLEAN(stored=True),
    has_video=BOOLEAN(stored=True),
    category=TEXT(stored=True), 
    place=TEXT(stored=True), 
    activity=TEXT(stored=True), 
    rating=NUMERIC(stored=True), )

In [2]:
import os.path
if not os.path.exists('index'):
    os.mkdir('index')
    index = create_in('index', schema)
else:
    index = open_dir('index')

In [3]:
from pandas import read_csv
posts_data = read_csv('./risk/posts_data.csv')
posts_data.head()

,Unnamed: 0,ID,author,date,img,rating,tag_activity,tag_category,tag_group,tag_place,tag_xtype,title,video
0,0,blog-215386.txt,/people/kate10,10.10.2018,True,25,Альпинизм,Новости,NaN,NaN,NaN,"Жетон ""Спасение в Горах"" снова в Москве",False
1,1,blog-215345.txt,/people/redakciyasajta,03.10.2018,True,77,Воздух,Взаимопомощь,NaN,NaN,NaN,Леониду Мартынову нужна помощь!,False
2,2,blog-215348.txt,/people/redakciyasajta,03.10.2018,True,73,Альпинизм,Новости,NaN,NaN,NaN,Виталий и Антон возвращаются!,False
3,3,blog-215335.txt,/people/robinsya,02.10.2018,True,88,Альпинизм,Помним,NaN,NaN,NaN,Марафон памяти – марафон дружбы,False
4,4,blog-215186.txt,/people/sergeyglaznov,05.09.2018,True,253,Альпинизм,Новости,NaN,NaN,NaN,Хочу заступиться за Пакистанцев!,False


In [4]:
path = '../posts_utf8/'
filenames = [name for name in os.listdir(path)]
def add_doc(writer, data, Id, activity, autor, title, img, video, category, date, place, rating):
    with open("../posts_utf8/"+Id, 'r', encoding='utf8') as f:
        text = f.read()
        writer.add_document(Id=Id, content=text, activity=activity, has_image=img, has_video=video,
                           title=title, author=author, date=date, place=place, category=category,
                            rating=rating)

In [5]:
from pandas import notnull
from datetime import datetime
from tqdm import tqdm

writer = index.writer()
for i, row in tqdm(posts_data.iterrows()):
    if row['ID'] not in filenames:
        continue
    activity = row['tag_activity'] if notnull(row['tag_activity']) else ''
    title = row['title'] if notnull(row['title']) else ''
    author = row['author'] if notnull(row['author']) else ''
    date = datetime.strptime(row['date'], "%d.%m.%Y")
    has_image = bool(row['img']) 
    has_video = bool(row['video'])
    category = row['tag_category'] if notnull(row['tag_category']) else '' 
    place = row['tag_place'] if notnull(row['tag_place']) else ''
    rating = int(row['rating'])
    add_doc(writer, posts_data, row['ID'], activity, author, title,has_image, 
            has_video, category, date, place, rating )
writer.commit()


33863it [06:58, 81.00it/s] 


In [22]:
from whoosh.query import *
myquery = And([Term('activity', u'альпинизм'),Term('content', u'амангельды'), Term('content', u'учитель')])
with index.searcher() as s:
    
    results = s.search(myquery)
    print(len(results))
    print(results[2]['content'])

6
Согласитесь, каждый любитель путешествий, спорта, экстрима, живущий по графику отпуска 28/365, задается насущно-рациональным вопросом, как и где провести драгоценные новогодние праздники. Не миновала чаша сия и наших участников DRC Extreme Club.
Меньшая, но прекрасная часть из нас, выбрала Северный Тань-Шань. Лирическо-эмоциональный рассказ о поездке на Заилийский Алатау (Казахстан) от Веры Романовой.
Как обычно принято у нас с друзьями, 31 декабря мы стараемся отмечать в Горах...
Было желание не только приятно, но и продуктивно провести русские зимние каникулы, ибо отпуска катастрофически не хватает. Рассматривались только те районы, где был бы выпускающий. Пару писем, и решение принято: альплагерь Туюк-Су, Казахстан.
Сборы снаряжения и мыслей в одну кучу занимают последние полтора часа до выхода на аэроэкспресс, стремящийся в Шереметьево.
Вопрос добраться до места не стоял – лагерь находится довольно близко к Алма-Ате, не больше 40 километров от аэропорта. Летели Air Astana c перев